In [1]:
import pandas as pd
import investpy as inv
from datetime import date
import datetime
import requests
import plotly.express as px
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
from dash.dependencies import Input, Output

C:\Users\Lucas Choconi\AppData\Local\Temp\ipykernel_14600\250894183.py:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Lucas Choconi\AppData\Local\Temp\ipykernel_14600\250894183.py:12: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
class HTMLTableParser:
    
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        #if len(column_names) > 0 and len(column_names) != n_columns:
        #    raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                            index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [3]:
#Lista de acciones del MERVAL
hp = HTMLTableParser()
table = hp.parse_url('https://bolsar.info/lideres.php')[0][1] # Grabbing the table from the tuple
stocksMerval = pd.DataFrame(table['Especie'])
stocksMerval = pd.concat([stocksMerval])
stocksMerval = stocksMerval.rename(columns={'Especie': 'Ticker'})
stocksMerval = stocksMerval.drop_duplicates()
List = stocksMerval.to_dict('split')['data']
List = [item for sublist in List for item in sublist]
stocksMerval = {'Ticker': List[:]}

#Armar una tabla de acciones del Merval
#Headers Tabla1 - Simbolo; Fecha; Precio; Valor ARP; Valor USD (info retrospectiva a 2 años)
#Armar una Tabla de CEDEARS y su acción en NYSE
#Headers Tabla2 - Simbolo, Mercado, Precio, paridad (X Cederas equivalen 1 acción en NYSE),  TC implicito, Orden de compra para arbitrar

dateToday = date.today()
dateToday = datetime.datetime.strptime(str(dateToday),'%Y-%m-%d').strftime('%d/%m/%Y')
fistDate = date.today()-relativedelta(years=3)
fistDate = datetime.datetime.strptime(str(fistDate),'%Y-%m-%d').strftime('%d/%m/%Y')

index = pd.date_range(dateToday,fistDate)
dataMerval = pd.DataFrame(index=index, columns=['Open', 'High', 'Low', 'Close', 'Volume',	'Currency',	'Ticker'])

for i in range(0,len(stocksMerval['Ticker'])):
  try:
    Ticker=stocksMerval['Ticker'][i]
    dataTicker = pd.DataFrame(inv.get_stock_historical_data(stock =Ticker, country = 'Argentina', from_date=fistDate, to_date= dateToday))
    dataTicker['Ticker'] = Ticker
    dataMerval = pd.concat([dataMerval, dataTicker])
  except:
    pass

dataMerval = dataMerval.drop(columns=['Open', 'High', 'Low', 'Volume', 'Currency']).dropna()
display ( dataMerval.head(2) )
display ( dataMerval.tail(2) )

c:\Users\Lucas Choconi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: UserWarning: Parsing '25/07/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\Lucas Choconi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: UserWarning: Parsing '25/07/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Close,Ticker
2019-07-25,17.2,ALUA
2019-07-26,17.5,ALUA


,Close,Ticker
2022-07-22,970.0,YPFD
2022-07-25,1046.0,YPFD


In [4]:
#Obtención del dólar CCL
argGGAL = pd.DataFrame(inv.get_stock_historical_data(stock ='GGAL', country = 'Argentina', from_date=fistDate, to_date= dateToday))
usdGGAL = pd.DataFrame(inv.get_stock_historical_data(stock ='GGAL', country = 'United States', from_date=fistDate, to_date= dateToday))
stockGGAL =  pd.DataFrame(pd.merge(argGGAL, usdGGAL, how='left', on=['Date'])).dropna()  
stockGGAL['Index'] = range(1, len(stockGGAL)+1)
stockGGAL = stockGGAL.set_index('Index', append=True).reset_index(level=0)  
stockGGAL ['USD'] = stockGGAL['Close_x']*10 / stockGGAL['Close_y']
histCCL = stockGGAL.drop(columns=['Close_y','Close_x','Open_x', 'High_x', 'Low_x','Volume_x', 'Currency_x','Open_y', 'High_y', 'Low_y','Volume_y', 'Currency_y'])
display ( histCCL.head(2) )
display ( histCCL.tail(2) )

,Date,USD
Index,,
1,2019-07-25,43.523697
2,2019-07-26,43.292357


,Date,USD
Index,,
713,2022-07-22,327.922078
714,2022-07-25,331.269350


In [5]:
mervalARG = pd.DataFrame(inv.get_index_historical_data(index ='S&P Merval', country = 'Argentina', from_date=fistDate, to_date= dateToday))
mervalARG = mervalARG.drop(columns=['Open', 'High', 'Low', 'Volume', 'Currency']).dropna()
mervalARG['Ticker'] = 'MERVAL'
dataMerval = pd.concat([dataMerval, mervalARG])
dataMerval.index.name = 'Date'
dataMerval = pd.DataFrame(pd.merge(histCCL, dataMerval, how='left', on=['Date'])).dropna() 
dataMerval ['Close USD'] = dataMerval['Close']/dataMerval['USD']
dataMerval
stocksMerval["Ticker"].append("MERVAL")

In [6]:
CCL = "Fx Rate (CCL): "+str(round(histCCL["USD"].iloc[-1], ndigits=2))

external_stylesheets = [dbc.themes.ZEPHYR]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = "Merval"
app.layout = html.Div([
                        html.H1(
                            "Historical Data - S&P Merval",
                            style={'text-align': 'left', 'background-color':'#0466C8', 'color':'#EBEBEB', "margin-bottom": "15px" ,  "padding": "10px"}
                            ),

                        html.H2(
                            CCL,
                            style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                            ),
                        
                        dbc.Row(
                            [
                                dbc.Col(
                                    html.H3(
                                    "Select a ticker:",
                                    style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                                    ),
                                    width="auto"),

                                dbc.Col(
                                    dcc.Dropdown(
                                        id="Select_Ticker",                    
                                        options=[{'label': x, 'value': x} for x in stocksMerval['Ticker']],
                                        multi=False,
                                        value='MERVAL',
                                        style={'width': "40%", 'color': '#9B9C9B'}
                                        ),
                                    width="10"
                                    )
                            ],
                        ),



                        dbc.Row(
                                [
                                    dbc.Col(

                                        dcc.Graph(id='lines_plot_USD',config={"displayModeBar": True}, figure={}),
                                        width="auto"),
                                
                                    dbc.Col(
                                        dcc.Graph(id='lines_plot_ARP',config={"displayModeBar": True}, figure={}),
                                        width="auto"),
                                ],
                                justify="evenly",
                                ),
                        ])
                      
                        
@app.callback(
    dash.dependencies.Output('lines_plot_USD', 'figure'),
    dash.dependencies.Output('lines_plot_ARP', 'figure'),
    [dash.dependencies.Input('Select_Ticker', 'value')])

def update_output(value):
    dff = dataMerval[dataMerval["Ticker"] == value]

    fig1 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close USD"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.1f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "USD",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "xaxis": {"fixedrange": True},
                                                  "yaxis": {"tickprefix": "$", "fixedrange": True},
                                                  "colorway": ["#007BBA"],
                                                  "width": 5,
                                                  "height": 5,
                                              },
                                          }
    fig2 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.0f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "ARS (Argentine Peso)",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "xaxis": {"fixedrange": True},
                                                  "yaxis": {"tickprefix": "$", "fixedrange": True},
                                                  "colorway": ["#7CCDF4"],
                                                  "width": 5,
                                                  "height": 5,
                                              },
                                          }    

    return fig1, fig2
                              
              
app.run_server(mode='inline')